In [12]:
##part 1
##use models with default
##use data set with H/A +1, -1
##do full window for now

##next:
##check if just excluding first 10 days helps (chaotic)
##check if different windows help

##next
## can try tuning (for loops by hand, or ... use grid_search (use ML mastery code))
##-I think tuning will be faster ... just do by hand ... loop over the possible things 
##-ONE for loop over i = (a,b,c,d)... for each model i[0]

##Orrr can try adding features ... here we have to worry about:
##-adding basic features eg pp, and correct fo%
##-scaling numericals
##-dummy vars for categoricals (are there any?) besides H/A
##-num_windows and which lengths for moving avgs
##-filtering the features for increasing complexity inteligently
##-There is a dicotemy: 
##(a)use H/A + numerics or  ... here I think it can be made more like time-series
##(b) just use mumerics (moving avg) ... here I think the order of the games is not important (note Leung did this, and random train)



In [ ]:
Pischsada features #variation of Ottawa

Goals for Total number of goals scored so far this season

Goals against

Total number of goals conceded so far thisseason

Goals Differential Goals for – Goals against

Power Play Success Rate Ratio – scoring a goal when 5 on 4

Power Kill Success Rate Ratio – not conceding a goal when 4 on 5

Shot % Goal scored/shots takenaaa

Save % Goals conceded/shots saved

Winning Streak Number of consecutive games won

Conference Standing Latest ranking on conference table

Fenwick Close % Possession ratio

PDO Luck parameter

5/5 Goal For/Against Ratio – 5 on 5 Goals For/Against

In [5]:
##Leung features (medium betting article)

feat_Leung = ['ID', 
               'Date', 
               'HomeTeam', 
               'AwayTeam', 
               'CF%', 
               'CSh%',
               'CSv%',
               'FF%',
               'FSh%',
               'FSv%',
               'GDiff',
               'GF%',
               'PDO',
               'PENDiff',
               'SF%',
               'SDiff',
               'Sh%',
               'Sv',
               'FOW%',
               'W%',
               'FavoritesW%',
               'Result']

In [ ]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak', 'pts%', 'win%', 'Fclose%', 'PDO'] 

In [ ]:
Pischsada features #variation of Ottawa

Goals for Total number of goals scored so far this season

Goals against

Total number of goals conceded so far thisseason

Goals Differential Goals for – Goals against

Power Play Success Rate Ratio – scoring a goal when 5 on 4

Power Kill Success Rate Ratio – not conceding a goal when 4 on 5

Shot % Goal scored/shots takenaaa

Save % Goals conceded/shots saved

Winning Streak Number of consecutive games won

Conference Standing Latest ranking on conference table

Fenwick Close % Possession ratio

PDO Luck parameter

5/5 Goal For/Against Ratio – 5 on 5 Goals For/Against

In [6]:
##note KNN or other clusters might be helpful group the teams in smart way ... but not now.
#models

##regression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

#classifiers (non-tree)
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


#tree-based classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor

##regression models
lr = Ridge(alpha=0.001) 
rfr = RandomForestRegressor(max_depth=3, random_state=0)
xgbr = XGBRegressor()

##classifier models
lrc = RidgeClassifier()
gnb = GaussianNB()
lgr = LogisticRegression(random_state = 0)
svc = SVC()

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=3, random_state=0)
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
xgbc = XGBClassifier()



##hyper_parameters from here 
##https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/
##for xgboost from here 
##https://machinelearningmastery.com/extreme-gradient-boosting-ensemble-in-python/

#xgb

trees = [10, 50, 100, 500, 1000, 5000]  #100  #num of trees
max_depth = range(1,11)  ##3-5
rates = [0.0001, 0.001, 0.01, 0.1, 1.0]  #0.1
subsample in arange(0.1, 1.1, 0.1):  #0.4, 0.5  ##this is 0.1, 0.2 ... 1.0 # % of features to sample


#svc 
kernels in [‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’] #if you use poly, then adjust degree
C in [100, 10, 1.0, 0.1, 0.001]

#gb

learning_rate in [0.001, 0.01, 0.1]
n_estimators [10, 100, 1000]
subsample in [0.5, 0.7, 1.0]
max_depth in [3, 7, 9]


#rfc
max_features [1 to 20]  #key
max_features in [‘sqrt’, ‘log2’]
n_estimators in [10, 100, 1000]

#bc
n_estimators in [10, 100, 1000]

svm_dic = {'kernels':[‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’]}
lrc_dic = {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
lgr_hp_dic = {'solver': [‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’], 'penalty' : [‘none’, ‘l1’, ‘l2’, ‘elasticnet’],
'C' :[100, 10, 1.0, 0.1, 0.01]}

In [7]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score


import matplotlib.pyplot as plt
import seaborn as sns


##couple evaluation functions ##removed model_name as variable
def evaluate_binary_classification(y_test, y_pred, y_proba=None, graph = False):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    #try:
    if y_proba != None:
        rocauc_score = roc_auc_score(y_test, y_proba)
    else:
        rocauc_score = "no roc"
    #except: 
    #    pass     
    cm = confusion_matrix(y_test, y_pred)
    if graph == True:
        sns.heatmap(cm, annot=True)
        plt.tight_layout()
        plt.title(f'{model_name}', y=1.1)
        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')
        plt.show()
    print("accuracy: ", accuracy)
    print("precision: ", precision)
    print("recall: ", recall)
    print("f1 score: ", f1)
    print("rocauc: ", rocauc_score)
    print(cm)
    #return accuracy, precision, recall, f1, rocauc_score

def evaluate_regression(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print("mae", mae)
    print("mse", mse)
    print('r2', r2)
    
##display null values


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data


In [8]:

data = pd.read_csv("/Users/joejohns/data_bootcamp/GitHub/final_project_nhl_prediction/Data/Shaped_Data/data_bet_stats_mp.csv")
data.drop(columns=[ 'Unnamed: 0'], inplace=True)

In [9]:
data['won'] = data['won'].apply(int)
data_playoffs = data.loc[data['playoffGame'] == 1, :].copy()  #set aside playoff games ... probably won't use them.
data=  data.loc[data['playoffGame'] == 0, :].copy() 

#sorted(data.columns)

In [10]:
def get_dates(X, season):  #season 20162017 int; returns the list of all dates in chronological order from that season 20161004, 20161005, ...
    X = X.loc[X['season'] == season, :].copy()
    dates_1 = list(set(X.loc[(X['date'] >= 900) & (X['date'] <= 1231) , :]['full_date']))  #2016 part
    dates_2 = list(set(X.loc[(X['date'] >= 100) & (X['date'] <= 800) , :]['full_date']))  #2017 part
    dates = dates_1 + dates_2 #all dates in order
    return dates

#sorted(data.columns)

In [11]:


def make_HA_diff(X, season, list_var_names = None ):
    X = X.loc[X['season'] == season, :].copy()
    X_H = X.loc[X['HoA'] == 'home',:].copy()
    X_A = X.loc[X['HoA'] == 'away',:].copy()
    X_H['goal_difference'] = X_H['goalsFor'] - X_H['goalsAgainst']  ##note every thing is based in home data
    X_H.reset_index(drop = True, inplace = True)
    X_A.reset_index(drop = True, inplace = True)
    df_visitor = pd.get_dummies(X_H['nhl_name'], dtype=np.int64)
    df_home = pd.get_dummies(X_A['nhl_name'], dtype=np.int64)
    df_model = df_home.sub(df_visitor) 
    df_model['date'] = X_H['date']
    df_model['full_date'] = X_H['full_date']
    
    df_model['game_id'] = X_H['game_id']
    df_model['home_id'] = X_H['team_id']
    df_model['away_id'] = X_A['team_id'] 
    y = X_H.loc[:,['date', 'full_date','game_id', 'Open','goal_difference', 'won']].copy()   ##these are from home team perspective; 'Open' is for betting 
    return (df_model, y)

##try later maye 
def make_HA_concat(X, season, list_var_names = None ):
    X = X.loc[X['season'] == season, :].copy()
    X_H = X.loc[X['HoA'] == 'home',:].copy()
    X_A = X.loc[X['HoA'] == 'away',:].copy()
    X_H['goal_difference'] = X_H['goalsFor'] - X_H['goalsAgainst']  ##note every thing is based in home data
    X_H.reset_index(drop = True, inplace = True)
    X_A.reset_index(drop = True, inplace = True)
    df_visitor = pd.get_dummies(X_H['nhl_name'], dtype=np.int64)
    df_home = pd.get_dummies(X_A['nhl_name'], dtype=np.int64)
    
    
    df_model = df_home.sub(df_visitor) 
    
    
    df_model['date'] = X_H['date']
    df_model['full_date'] = X_H['full_date']
    
    df_model['game_id'] = X_H['game_id']
    df_model['home_id'] = X_H['team_id']
    df_model['away_id'] = X_A['team_id'] 
    y = X_H.loc[:,['date', 'full_date','game_id', 'Open','goal_difference', 'won']].copy()   ##these are from home team perspective; 'Open' is for betting 
    return (df_model, y)


In [12]:
##have not written yet

def top3_max_val_params(model, X, dates, drop_firs=False):
    pass
    


In [13]:
#this is for  regressors predicting wins - losses, can use this to turn output into win prediction 

def make_win(x):
    if x <= 0:
        return 0
    if x >0:
        return 1

v_make_win = np.vectorize(make_win)

#useage: v_make_win(y_pred)

In [14]:
def regr_model_results(model, model_name, X, dates, step, window_size, prediction_size, drop_first_k_days = 0): #X = data 
    results_dic['model_name'] = []
    results_dic['date'] = []
    results_dic['mae'] = []
    results_dic['mse'] = []
    results_dic['r2'] = []   
    
    
    #drop first k days from dates and X
    dates = dates[drop_first_k_days :]
    X = X.loc[X['full_date'].isin(dates), :].copy()  

    for i in range(step, len(dates), step): ##eg step =10, so 17 rounds
        
        model.fit(X.loc[X['full_date'].isin(dates[max(i-window_size ,0):i]), :],y.loc[y['full_date'].isin(dates[max(i-window_size,0):i]),'goal_difference' ])
        y_pred = model.predict(X.loc[X['full_date'].isin(dates[i:i+prediction_size]), :])
        y_test = y.loc[y['full_date'].isin(dates[i:i+prediction_size]),'goal_difference' ]
        
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results_dic['model_name'].append(model_name)
        results_dic['date'].append(dates[i])
    
        results_dic['mae'].append(mae)
        results_dic['mse'].append(mse)
        results_dic['r2'].append(r2)
        
    return results_dic #!
        
        

In [15]:
def class_model_results(model, model_name, X, dates, step, window_size, prediction_size, drop_first_k_days = 0): #X = data 
    results_dic ={}
    results_dic['model_name'] = []
    results_dic['date'] = []
    results_dic['accuracy'] = []
    results_dic['f1_score'] = []

    #results_dic['precision'] = []
  #  results_dic['recall'] = []
    
    #drop first k days from dates and X
    dates = dates[drop_first_k_days :]
    X = X.loc[X['full_date'].isin(dates), :].copy()  

    for i in range(step, len(dates), step): ##eg step =10, so 17 rounds
        model.fit(X.loc[X['full_date'].isin(dates[max(i-window_size ,0):i]), :],y.loc[y['full_date'].isin(dates[max(i-window_size,0):i]),'won' ])
        y_pred = model.predict(X.loc[X['full_date'].isin(dates[i:i+prediction_size]), :])
        y_test = y.loc[y['full_date'].isin(dates[i:i+prediction_size]),'won' ]
    
        accuracy = accuracy_score(y_test, y_pred)
        #recision = precision_score(y_test, y_pred, zero_division = 0)
        #recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred) #, average = None)
            
        results_dic['model_name'].append(model_name)  #append same model name every iter so same length as others
        results_dic['date'].append(dates[i])
                          
        results_dic['accuracy'].append(accuracy)
        results_dic['f1_score'].append(f1)
        #results_dic['precision'].append(precision)
        #results_dic['recall'].append(recall)
    results_dic['model_name'].append('model_name'+'_avg')  #append same model name every iter so same length as others
    results_dic['date'].append('average')
    results_dic['accuracy'].append(round(np.mean(np.array(results_dic['accuracy'])), 2) ) 
    results_dic['f1_score'].append(round(np.mean(np.array(results_dic['f1_score'])), 2) ) 
    return results_dic #!
        
        

In [17]:
all_seasons = sorted(set(data['season']))
#all_seasons

1. two versions of dummies for teams - teams plus H/A and teamH - teamA 
2. play with add/remove when in season (hypothesis: offense early defense favored late) ---> 5 vars for where in year ... 

In [20]:
"dajfdajjj"[:-3]

'dajfda'

In [22]:
data['faceOffWinPercentage'] = 

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
29933    50.9
29934    53.3
29935    46.7
29936    48.2
29937    51.8
Name: faceOffWinPercentage, Length: 28358, dtype: float64

In [26]:
def make_diff(statFor, statAway):
    return statFor - statAway

v_make_diff = np.vectorize(make_diff)

def make_per(statFor, statAway):  #example FOWFor/(FOWFor + FOWAgainst)  or ShAt
    return statFor/(statFor+statAway)

v_make_per = np.vectorize(make_per)

##have to adjust later depending what is convenient to use for stat name ... 

#not using k_days_back .. will use a unioversal fn to just avg past games...
#makes more sense ,,, if they get a 1000 shots against one game, don't want that to leak into 
#other games .. 
def get_per(X, stat_name, k_days_back = 1000  ): #do max(current -k, 0) so this will be all games
    #stat_name = goalsFor or faceoffsWonFor  example to keep in mind ... mp style
    ##we do this so we can loop thru existing names in our feature list
    stat_name = stat_name[:-3]  #remove last 3
    statFor = stat_name+'For'
    statAgainst = stat_name+'Against'
    return v_make_per(X[statFor],X[statAgainst])


def get_diff(X, stat_name, k_days_back = 1000  ): #do max(current -k, 0) so this will be all games
    #stat_name = goalsFor or faceoffsWonFor  example to keep in mind ... mp style
    ##we do this so we can loop thru existing names in our feature list
    stat_name = stat_name[:-3]  #remove last 3
    statFor = stat_name+'For'
    statAgainst = stat_name+'Against'
    return v_make_diff(X[statFor],X[statAgainst])
    
    

In [38]:
[1,3,4, 7, 8][-3:]

[4, 7, 8]

In [120]:
k = 10

X_16 = data.loc[data['season'] == 20162017, :].copy()
all_dates = sorted(list(X_16['full_date']))

date1 = 20161022
dates_bef_date1 = [date for date in all_dates if date < date1]
dates_bef_date1 = dates_bef_date1[-k :]

stat_name = 'goalsFor'
sk = str(k)
X_16[stat_name+sk+'_avg'] = np.NaN 

X_bef_date1 = X_16.loc[X_16['full_date'].isin(dates_bef_date1), :]
teams_date1 = sorted(list(X_bef_date1['nhl_name']))
nhl_name = 'ANA' 
X = X_16.copy()
k_avg = np.mean(X_bef_date1.loc[(X_bef_date1['nhl_name'] == nhl_name) , stat_name])
X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+sk+'_avg'] =k_avg
k_avg    

3.0

In [131]:
all_dates = sorted(set((X_16['full_date'])))

X_16.iloc[0,2]

114

In [142]:
 #num of games back max(xurrent - k, 0) = 0 for 1000
     #here X_N = data.iloc[0:date-1, :] all games from all days before present day.  
    #here X = whole season or whatever chumk you want
def get_k_game_avg(data, stat_name, k_days_back= 3000 ): #k_days_back
    X = data.copy()
    k = str(k_days_back)
    all_dates = sorted(set(X['full_date']))
    X[stat_name+k+'_avg'] = np.NaN  #set up column
    
    #take care of first day of season ... might want to leave it NaN ... nahhh
    date0 = all_dates[0]
    X.loc[(X['full_date'] == date0), stat_name+k+'_avg'] =  X.loc[(X['full_date'] == date0), stat_name]
    
    for date1 in all_dates[1:]: #all but first date so don't get emopty
        dates_bef_date1 = [date for date in all_dates if date < date1]
        dates_bef_date1 = dates_bef_date1[-k_days_back :] # this grabs all or the last k days of list
        
        X_bef_date1 = X.loc[X['full_date'].isin(dates_bef_date1), :]
        teams_date1 = sorted(set(X_bef_date1['nhl_name']))
        
        #first take care of all teams not present in the previous days (only a few)
        for nhl_name in set(X['nhl_name']).difference(set(X_bef_date1['nhl_name'])):
            k_avg = X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name] ##should be single number anyway
            X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+k+'_avg'] =k_avg
        
        
        #this is the main step
        for nhl_name in teams_date1:
            k_avg = np.mean(X_bef_date1.loc[(X_bef_date1['nhl_name'] == nhl_name) , stat_name])
            X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+k+'_avg'] =k_avg
    return X

In [143]:
##I do not the Emperor's prize damaged. We will test it on Captain Solo.

In [144]:
get_k_game_avg(X_16, 'goalsFor', 10)


#data['faceOffWinPercentage'] = get_per(data, 'faceOffsWonFor')
 

In [145]:

perc_null(X_16.loc[:, ['full_date', 'team_id', 'goalsFor', 'goalsFor10_avg', 'goalsFor20_avg']])

,Total,Type,Percent
goalsFor20_avg,8,float64,0.003252
full_date,0,int64,0.000000
team_id,0,int64,0.000000
goalsFor,0,float64,0.000000
goalsFor10_avg,0,float64,0.000000


In [147]:
#X_16.loc[:, ['full_date', 'nhl_name', 'goalsFor', 'goalsFor10_avg', 'goalsFor20_avg']].iloc[0:20, :]

In [29]:
data['faceOffWinPercentage']  #this is for each game only. I think it is ok to just avg these ... ffs

0        0.483871
1        0.516129
2        0.466667
3        0.533333
4        0.630435
           ...   
29933    0.508772
29934    0.533333
29935    0.466667
29936    0.222222
29937    0.777778
Name: faceOffWinPercentage, Length: 28358, dtype: float64

In [152]:
(X_16['penaltiesAgainst'] == X_16['powerPlayOpportunities']).sum()

1152

In [ ]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 

In [156]:
#I add 'pts%', 'win%', 'xgoal%', ('Fsh%' ...), grouped_4 date? 

##Leung features (medium betting article) same as Morrison pretty much and other guy not much diff ... 

set(['ID', 
          'Date', 
               'HomeTeam', 
               'AwayTeam', 
               'CF%', 
               'CSh%',
               'CSv%',
               'FF%',
               'FSh%',
               'FSv%',
               'GDiff',
               'GF%',
               'PDO',
               'PENDiff',
               'SF%',
               'SDiff',
               'Sh%',
               'Sv',
               'FOW%',
               'W%',
               'FavoritesW%',
               'Result'])

{'AwayTeam',
 'CF%',
 'CSh%',
 'CSv%',
 'Date',
 'FF%',
 'FOW%',
 'FSh%',
 'FSv%',
 'FavoritesW%',
 'GDiff',
 'GF%',
 'HomeTeam',
 'ID',
 'PDO',
 'PENDiff',
 'Result',
 'SDiff',
 'SF%',
 'Sh%',
 'Sv',
 'W%'}

In [ ]:
feat_Pis_plus = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak', 'pts%', 'win%', 'Fclose%', 'PDO'] 

In [153]:
#features I need to do this ...
feat_for_Pis = ['full_date','season', 'game_id', 

'nhl_name','HoA',
 'opposingTeam', 

  'goalsAgainst',
 'goalsFor',
'powerPlayGoals',
 'powerPlayOpportunities',     
 
'shotsOnGoalAgainst',
  'shotsOnGoalFor',
'savedShotsOnGoalAgainst',
 'savedShotsOnGoalFor',  

'fenwickPercentage',
  
#'win_streak_grouped_10', 'conference_standing_grouped_10',
#'PDO'  

#targets 
  'won',
 'settled_in',] #could later try to train classifier to predict this ... tough tho --> reg or no
    

extra = ['corsiPercentage', 
    'penaltiesAgainst',
 'penaltiesFor', 
    'shotAttemptsAgainst',
 'shotAttemptsFor',
    'unblockedShotAttemptsAgainst',
 'unblockedShotAttemptsFor', 'savedUnblockedShotAttemptsAgainst',
 'savedUnblockedShotAttemptsFor',
 'xGoalsPercentage','scoreVenueAdjustedxGoalsAgainst',
 'scoreVenueAdjustedxGoalsFor','blockedShotAttemptsAgainst',
 'blockedShotAttemptsFor',

 'flurryAdjustedxGoalsAgainst',
 'flurryAdjustedxGoalsFor',
 'flurryScoreVenueAdjustedxGoalsAgainst',
 'flurryScoreVenueAdjustedxGoalsFor',
 'missedShotsAgainst',
 'missedShotsFor',]



In [ ]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 





